<a href="https://colab.research.google.com/github/mb16biswas/llm/blob/main/ensemble_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install langchain-huggingface
!pip install huggingface_hub
# !pip install langchain-huggingface==0.1.1
# !pip install huggingface-hub==0.27.1
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes
!pip install langchain-community langchain-core
!pip install --upgrade langchain

!pip install datasets==2.16.1
!pip install scikit-learn numpy pandas
!pip install transformers
!pip install -q -U trl accelerate git+https://github.com/huggingface/peft.git
!pip install -q bitsandbytes einops sentencepiece
!pip install evaluate seqeval
!pip uninstall -y apex



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [10]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
import pandas as pd
import string
from langchain.docstore.document import Document
import argparse
import re
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline
from transformers import AutoModel,AutoModelForMaskedLM
from typing import List
import torch
import torch.nn.functional as F

from torch import Tensor
import os
from transformers import set_seed
from huggingface_hub import login
import time
import argparse
import datasets
from datasets import load_dataset, Dataset

In [3]:
sec_key =
os.environ["HUGGINGFACEHUB_API_TOKEN"]=sec_key
login(token = sec_key)

In [4]:
flag = 5

if(flag == 1):

    repo_id = "mistralai/Mistral-7B-v0.3"

elif(flag == 2):

    repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

elif(flag == 3):

    repo_id = "meta-llama/Llama-2-7b-chat-hf"

elif(flag == 4):

    repo_id = "meta-llama/Meta-Llama-3-8B-Instruct"


elif(flag == 5):

    repo_id = "google/gemma-2b-it"


elif(flag == 6):

    repo_id = "google/gemma-7b-it"

In [5]:
f2 = 2


if(f2 == 1):

    llm=HuggingFaceEndpoint(repo_id=repo_id,max_length=2048,token=sec_key,task="text-generation")

else:

    tokenizer = AutoTokenizer.from_pretrained(repo_id, device_map= "auto")
    model = AutoModelForCausalLM.from_pretrained(repo_id, device_map="auto", torch_dtype=torch.float16)
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=2048, return_full_text=False, temperature=0.1)
    llm = HuggingFacePipeline(pipeline=pipe, batch_size=1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-5-3596895178>:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe, batch_size=1)


In [8]:
d_name = ['covidqa', 'cuad', 'delucionqa', 'emanual', 'expertqa', 'finqa', 'hagrid', 'hotpotqa', 'msmarco', 'pubmedqa', 'tatqa', 'techqa']

In [11]:
for d in d_name:

    data = load_dataset("rungalileo/ragbench", d)

    train_ = data["train"]
    val_ = data["validation"]

    break


README.md:   0%|          | 0.00/24.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/854k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/913k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1252 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/246 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/267 [00:00<?, ? examples/s]

In [38]:
def generate_prompt(context, question):
    prompt = f"""
You are a helpful assistant. Given a question and a context passage, your task is to generate a precise answer only if the answer is clearly present in the provided context.

If the answer is not found or not clearly present in the context, respond with:
"I don't know" or "The answer is not present in the context."

Additionally, provide a confidence score between 0.0 (no confidence) to 1.0 (full confidence) based on how certain you are that your answer is correct and supported by the context.

Return your response as a Python dictionary in the following format:

{{
  "answer": "<your generated answer>",
  "score": <confidence_score>
}}

Context:
{context}

Question:
{question}
"""
    return prompt





def generate_prompt(context, question):
    prompt = f"""
You are a helpful assistant. Your task is to answer a question based on a given context.

You must return two things:
1. A concise answer to the question.
2. A confidence score between 0.0 and 1.0 that reflects how confident you are in your answer being correct — whether the answer is present in the context or not.

### Guidelines:
- If the answer is **clearly stated in the context**, answer it directly and return a **high confidence score (e.g., 0.9–1.0)**.
- If the answer is **not present**, but you are confident it is not present, respond accordingly and still return a **high confidence score** (e.g., 0.9–1.0).
- If you are **unsure** or the information is ambiguous, return an appropriate answer (e.g., "I don't know") and a **lower confidence score** (e.g., 0.0–0.6).

Always respond in the following format:
```python
{{
  "answer": "<your generated answer>",
  "score": <confidence_score>
}}


### Examples:

Example 1
Context: The 2003 SARS outbreak was caused by a coronavirus known as SARS-CoV. It was believed to have originated in civet cats.
Question: What animal is thought to have originated the 2003 SARS outbreak?
Output:
{{
"answer": "civet cats",
"score": 0.95
}}

Example 2
Context: The passage discusses treatments for COVID-19 including Remdesivir and dexamethasone. It does not mention Favipiravir.
Question: Is Favipiravir mentioned as a treatment in the passage?
Output:
{{
"answer": "The answer is not present in the context.",
"score": 0.98
}}

Example 3
Context: The Nile River is over 6,600 kilometers long and flows through multiple countries including Egypt and Sudan.
Question: How many countries does the Nile flow through?
Output:
{{
"answer": "I don't know",
"score": 0.3
}}
---

### Now Answer This:

Context: {context}

Question: {question}
"""
    return prompt





In [39]:
i = 0
Ans = []
P = []

for t in train_:

    context = "".join(t["documents"])
    question = t["question"]

    p = generate_prompt(context, question)

    ans = llm.invoke(p)
    Ans.append(ans)
    P.append(p)

    i += 1

    if(i == 20):
        break

In [41]:
Ans

['Answer: The passage suggests that a lower T-cell count may be a risk factor for severe HAdV-55 infection.',
 'Answer: The passage does not specify what MVO is, so I cannot answer this question from the context.',
 'Answer:\n{\n"answer": "15 sera collected in 2012 to 2013 from 10,009 people in 13 KSA provinces contained MERS-CoV antibodies, but significantly higher proportions in occurred in camel shepherds and slaughterhouse workers.",\n"score": 0.92\n}',
 'Answer: The passage does not specify how untreated MDA-MB-231 cells were labeled, so I cannot answer this question from the provided context.',
 'Answer: 850,000\nConfidence score: 0.95',
 'Answer: The context does not provide the genome sequence that was available for this study, so I cannot answer this question from the provided context.',
 'Answer: The passage does not specify what destabilizes the system, so I cannot answer this question from the provided context.',
 'Answer: 1,255\n\nConfidence score: 0.95',
 'Answer: The con

In [48]:
import re
import ast

def extract_answers_and_scores(responses):
    extracted = []

    for item in responses:
        item = item.strip()

        # Case 1: If JSON-style dict is embedded in the string
        try:
            json_match = re.search(r'\{.*?\}', item, re.DOTALL)
            if json_match:
                parsed = ast.literal_eval(json_match.group())
                answer = parsed.get("answer", "").strip()
                score = float(parsed.get("score", 0.0))
                extracted.append({"answer": answer, "score": score})
                continue
        except Exception:
            pass

        # Case 2: If in plain text format with "Answer:" and optional "Confidence score:"
        answer_match = re.search(r'Answer:\s*(.*?)(?:\n|$)', item, re.DOTALL)
        score_match = re.search(r'Confidence score:\s*([0-9.]+)', item)

        answer = answer_match.group(1).strip() if answer_match else item
        score = float(score_match.group(1)) if score_match else 0.0

        extracted.append({"answer": answer, "score": score})

    return extracted


def extract_answers_and_scores(responses):
    extracted = []

    for item in responses:
        item = item.strip()

        # Try extracting JSON-style first
        try:
            json_match = re.search(r'\{.*"answer"\s*:\s*".*?".*"score"\s*:\s*[\d.]+.*\}', item, re.DOTALL)
            if json_match:
                parsed = json.loads(json_match.group().replace('\n', ' '))
                extracted.append({
                    "answer": parsed.get("answer", "").strip(),
                    "score": float(parsed.get("score", 0.0))
                })
                continue
        except Exception:
            pass

        # Regex fallback: catch malformed dict-like string
        answer_match = re.search(r"'?answer'?\s*[:=]\s*['\"]?(.*?)['\"]?(,|$)", item)
        score_match = re.search(r"score\s*[:=]\s*([0-9.]+)", item)

        answer = answer_match.group(1).strip() if answer_match else item
        score = float(score_match.group(1)) if score_match else 0.0

        extracted.append({
            "answer": answer,
            "score": score
        })


    return extracted

In [49]:
responses = [
    "'answer' : hellooooooooooooo baby ,score: 0.89",
    'Answer: The passage suggests that a lower T-cell count may be a risk factor for severe HAdV-55 infection.',
    'Answer: The passage does not specify what MVO is, so I cannot answer this question from the context.',
    'Answer:\n{\n"answer": "15 sera collected in 2012 to 2013 from 10,009 people in 13 KSA provinces contained MERS-CoV antibodies, but significantly higher proportions in occurred in camel shepherds and slaughterhouse workers.",\n"score": 0.92\n}',
    # (add more of your strings here)
]

result = extract_answers_and_scores(responses)
for r in result:
    print(r)

{'answer': 'hellooooooooooooo baby', 'score': 0.89}
{'answer': 'Answer: The passage suggests that a lower T-cell count may be a risk factor for severe HAdV-55 infection.', 'score': 0.0}
{'answer': 'Answer: The passage does not specify what MVO is, so I cannot answer this question from the context.', 'score': 0.0}
{'answer': 'Answer:\n{\n"answer": "15 sera collected in 2012 to 2013 from 10,009 people in 13 KSA provinces contained MERS-CoV antibodies, but significantly higher proportions in occurred in camel shepherds and slaughterhouse workers.",\n"score": 0.92\n}', 'score': 0.0}
